In [1]:
import os
%pwd

'c:\\Project\\MLops_basics_project\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Project\\MLops_basics_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float 
    target_column : str



In [5]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories


In [6]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
         
        create_directories([config.root_dir])
        model_trainer_config=ModelTrainerConfig(root_dir=config.root_dir,train_data_path=config.train_data_path,
                                                test_data_path=config.test_data_path,
                                                model_name=config.model_name,
                                                alpha=params.alpha,
                                                l1_ratio=params.l1_ratio,
                                                target_column=schema.name
                                                )
        return model_trainer_config






In [11]:
from box import ConfigBox

In [12]:
!pip show python-box


Name: python-box
Version: 7.3.2
Summary: Advanced Python dictionaries with dot notation access
Home-page: https://github.com/cdgriffith/Box
Author: Chris Griffith
Author-email: chris@cdgriffith.com
License: MIT
Location: C:\Users\Mayank\anaconda3\Lib\site-packages
Requires: 
Required-by: 


In [14]:
import pandas as pd
import os
from src.mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

""" 
joblib is a Python library used for efficiently serializing (saving) and deserializing (loading) Python objects, especially large data like NumPy arrays and machine learning models. It is commonly used in machine learning workflows to save trained models to disk and load them later for inference or further training.

Key uses:

Save and load scikit-learn models (joblib.dump, joblib.load)
Faster than pickle for large numerical data
"""

' \njoblib is a Python library used for efficiently serializing (saving) and deserializing (loading) Python objects, especially large data like NumPy arrays and machine learning models. It is commonly used in machine learning workflows to save trained models to disk and load them later for inference or further training.\n\nKey uses:\n\nSave and load scikit-learn models (joblib.dump, joblib.load)\nFaster than pickle for large numerical data\n'

In [17]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]
        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        

In [18]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-08-09 12:39:52,251: INFO: common: yaml file: C:\Project\MLops_basics_project\config\config.yaml loaded successfully]
[2025-08-09 12:39:52,253: INFO: common: yaml file: C:\Project\MLops_basics_project\params.yaml loaded successfully]
[2025-08-09 12:39:52,257: INFO: common: yaml file: C:\Project\MLops_basics_project\schema.yaml loaded successfully]
[2025-08-09 12:39:52,258: INFO: common: created directory at: artifacts]
[2025-08-09 12:39:52,260: INFO: common: created directory at: artifacts/model_trainer/]
